In [ ]:
import numpy as np

from weldx import Q_, U_, GenericSeries, MathematicalExpression, Time, TimeSeries

In [ ]:
class newTS(GenericSeries):

    # --- Constraints

    def preprocess_time(self, time, time_unit):
        time = Time(time)
        time_interp = Time(time, self._reference_time)
        return dict(t=time_interp.as_quantity().to(time_unit))

    _allowed_variables = ["t"]
    _required_dimensions = ["t"]
    _required_dimension_units = dict(t=U_("s"))
    _required_variables = ["t"]
    _evaluation_preprocessor = preprocess_time

    # --- Custom init

    def __init__(self, data, time=None, interpolation=None, reference_time=None):
        if time is not None:
            time = dict(t=Time(time, reference_time).as_quantity())

        self._reference_time = reference_time
        super().__init__(data=data, coords=time, interpolation=interpolation)

    # --- extra properties and functions

    @property
    def reference_time(self):
        return self._reference_time

    @property
    def time(self):
        if self.is_expression:
            return None
        t = self._data.coords["t"]
        return Time(Q_(t.data, t.attrs["units"]))  # There is probably a better way ;)

    def interp_time(self, time, time_unit="s"):
        return self(time=time, time_unit=time_unit)

## Discrete

### `__init__`

In [ ]:
try:
    TimeSeries(Q_([1, 3, 5], "m"), time=Q_([1, 2, 3], "K"))
except Exception as e:
    print(e)

In [ ]:
try:
    newTS(Q_([1, 3, 5], "m"), time=Q_([1, 2, 3], "K"))
except Exception as e:
    print(e)

In [ ]:
ts_discrete = TimeSeries(
    Q_([1, 3, 5], "m"), time=Q_([1, 2, 3], "s"), reference_time="2000-01-01"
)

In [ ]:
tsnew_discrete = newTS(
    Q_([1, 3, 5], "m"), time=Q_([1, 2, 3], "s"), reference_time="2000-01-01"
)

### attributes

In [ ]:
np.allclose(ts_discrete.data, tsnew_discrete.data)

In [ ]:
ts_discrete.data_array.equals(tsnew_discrete.data_array)

> Internal storage format differs (timedelta vs quantity). Also, the interpolation is yet not stored in the GS version

In [ ]:
ts_discrete.data_array

In [ ]:
tsnew_discrete.data_array

In [ ]:
ts_discrete.is_discrete == tsnew_discrete.is_discrete == True

In [ ]:
ts_discrete.is_expression == tsnew_discrete.is_expression == False

In [ ]:
ts_discrete.units == tsnew_discrete.units

In [ ]:
ts_discrete.interpolation == tsnew_discrete.interpolation

In [ ]:
ts_discrete.interpolation

In [ ]:
tsnew_discrete.interpolation

In [ ]:
np.all(ts_discrete.time == tsnew_discrete.time)

In [ ]:
ts_discrete.reference_time == tsnew_discrete.reference_time

> GS version is correct, TS has a bug :)

In [ ]:
ts_discrete.reference_time

In [ ]:
tsnew_discrete.reference_time

### interpolation

In [ ]:
ts_discrete.interp_time("3s")

In [ ]:
tsnew_discrete.interp_time("3s")

In [ ]:
d = tsnew_discrete.interp_time("3s")
d.data_array.data

In [ ]:
tsnew_discrete.data_array.sel(t=3).data

In [ ]:
np.allclose(ts_discrete.interp_time("3s").data, tsnew_discrete.interp_time("3s").data)

In [ ]:
np.allclose(
    ts_discrete.interp_time("2007").data, tsnew_discrete.interp_time("2007").data
)

> "Old TS" seems to be wrong here since it never got a proper support for absolute times. See below

In [ ]:
ts_discrete.interp_time("2007").data

In [ ]:
tsnew_discrete.interp_time("2007").data

In [ ]:
np.allclose(
    ts_discrete.interp_time(Q_([1, 2, 3], "s")).data,
    tsnew_discrete.interp_time(Q_([1, 2, 3], "s")).data,
)

## Expression

### `__init__`

#### only time parameter allowed

In [ ]:
try:
    TimeSeries(MathematicalExpression("a*t +b"))
except Exception as e:
    print(e)

In [ ]:
try:
    newTS(MathematicalExpression("a*t +b"))
except Exception as e:
    print(e)

#### units do not match cause t needs a time unit

In [ ]:
try:
    TimeSeries(MathematicalExpression("a*t +b", parameters=dict(a=5, b=3)))
except Exception as e:
    print(e)

In [ ]:
### TODO: _required_dimension_units should automatically update the required units
try:
    newTS(MathematicalExpression("a*t +b", parameters=dict(a=5, b=3)))
except Exception as e:
    print(e)

In [ ]:
ts_expr = TimeSeries(
    MathematicalExpression("a*t +b", parameters=dict(a="5/s", b=3)),
    reference_time="2000-01-01",
);

In [ ]:
tsnew_expr = newTS(
    MathematicalExpression("a*t +b", parameters=dict(a="5/s", b=3)),
    reference_time="2000-01-01",
);

### properties

In [ ]:
ts_expr.data == tsnew_expr.data

In [ ]:
ts_expr.data_array == tsnew_expr.data_array

In [ ]:
ts_expr.is_discrete == tsnew_expr.is_discrete == False

In [ ]:
ts_expr.is_expression == tsnew_expr.is_expression == True

In [ ]:
ts_expr.units == tsnew_expr.units

In [ ]:
# ts_expr.shape == ts_expr.shape

In [ ]:
# tsnew_expr.shape

In [ ]:
ts_expr.time == tsnew_expr.time

In [ ]:
ts_expr.reference_time == tsnew_expr.reference_time

In [ ]:
ts_expr.interpolation == tsnew_expr.interpolation

### interpolation

In [ ]:
ts_expr.interp_time("3s")

In [ ]:
tsnew_expr.interp_time("3s")

> TODO: Should return a new `TimeSeries` and not a `GenericSeries`

In [ ]:
np.allclose(ts_expr.interp_time("3s").data, tsnew_expr.interp_time("3s").data)

In [ ]:
np.allclose(ts_expr.interp_time("2007").data, tsnew_expr.interp_time("2007").data)

In [ ]:
np.allclose(
    ts_expr.interp_time(Q_([1, 2, 3], "s")).data,
    tsnew_expr.interp_time(Q_([1, 2, 3], "s")).data,
)